In [2]:
import xml.etree.ElementTree as ET
import requests
from io import StringIO

# Funktion zum Herunterladen und Parsen der XML-Datei von der URL
def fetch_and_parse_xml(url):
    response = requests.get(url)
    response.raise_for_status()  # Sicherstellen, dass die Anfrage erfolgreich war
    
    # Die XML-Daten als String einlesen und parsen
    xml_data = StringIO(response.text)
    tree = ET.parse(xml_data)
    return tree

# Funktion zum Extrahieren der relevanten Informationen
def extract_speech_data(tree):
    root = tree.getroot()

    speeches = []

    # Iteriere über alle "rede"-Elemente
    for rede in root.findall('.//rede'):
        # Extrahiere den Namen des Redners
        name = rede.find('.//name')
        speaker_name = ''
        party = ''
        
        if name is not None:
            first_name = name.find('vorname').text if name.find('vorname') is not None else ''
            last_name = name.find('nachname').text if name.find('nachname') is not None else ''
            speaker_name = f'{first_name} {last_name}'.strip()
            
            # Extrahiere die Parteizugehörigkeit
            party_element = name.find('fraktion')
            if party_element is not None:
                party = party_element.text

        # Extrahiere den Redetext, während Kommentare und Zwischenrufe ignoriert werden
        speech_text = []
        for p in rede.findall('.//p'):
            if not any(comment.tag == 'kommentar' for comment in p):
                speech_text.append(p.text.strip() if p.text else '')

        # Wenn der Redner oder die Partei fehlen, gibt es eine Möglichkeit,
        # dass sie an einem anderen Ort in der Struktur versteckt sind
        if not speaker_name and rede.find('redner') is not None:
            # Alternative Methode: Finde den Rednernamen direkt im "redner"-Tag
            speaker_name = rede.find('redner').text.strip() if rede.find('redner').text else ''
        
        speeches.append({
            'speaker': speaker_name,
            'party': party,
            'speech': ' '.join(speech_text)
        })

    return speeches

# Beispielaufruf
url = 'https://www.bundestag.de/resource/blob/1017784/ef2104603e7e88213110bd4d147d0d86/20185.xml'
tree = fetch_and_parse_xml(url)
speech_data = extract_speech_data(tree)

# Ergebnisse anzeigen
for speech in speech_data:
    print(f"Redner: {speech['speaker']}, Partei: {speech['party']}")
    print(f"Redeinhalt: {speech['speech']}\n")


Redner: Nancy Faeser, Partei: 
Redeinhalt:  Sehr geehrte Frau Präsidentin! Sehr geehrte Abgeordnete! Sehr geehrte Gäste! Vor knapp zwei Wochen erreichten uns die furchtbaren Nachrichten aus Solingen über die islamistische Tat, bei der drei Menschen brutal ermordet und acht Menschen schwerst verletzt wurden. Ich möchte zu Beginn dieser Debatte der Opfer gedenken und den Angehörigen der Opfer mein tief empfundenes Beileid aussprechen. Ich denke auch an die Verletzten. Herzliche Genesungswünsche von dieser Stelle. Meine Damen und Herren, diese furchtbaren und brutalen Morde durch einen Islamisten haben uns zutiefst erschüttert. Wir dürfen nach dieser Tat nicht einfach zur Tagesordnung übergehen, sondern müssen die notwendigen Konsequenzen daraus ziehen und als Bundesregierung, als Parlament für den Schutz der Bürgerinnen und Bürger das tun, was wir können, und das auch umsetzen. Das tun wir mit dem heutigen Gesetzespaket. Ich bin den Fraktionen von SPD, Bündnis 90/Grünen und der FDP sehr 